# Train a model and test it on the final test dataset

In [ ]:
from pathlib import Path
import time

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import model_from_json

# General settings
sns.set_style('whitegrid')

In [ ]:
load_from_web = True

In [ ]:
def load_data(filename):
    if load_from_web:
        url = 'https://raw.githubusercontent.com/Withington/deepscent/master/data/SonyAIBORobotSurface1_IoC/'+filename
        robot_df = pd.read_csv(url, sep='\t', header=None)
        print('Loaded from', url)
        robot_data = robot_df.values
    else:
        data_dir = '../../data'
        data_name = 'SonyAIBORobotSurface1_IoC'
        data_filename = data_dir+'/'+data_name+'/'+filename
        robot_data = np.loadtxt(Path(data_filename))
        print('Loaded from', data_filename)

    y = robot_data[:,0]
    x = robot_data[:,1:]
    print('The shape of x is', x.shape)
    print('The shape of y is', y.shape)

    # Change from classes 1 and 2 to classes 0 and 1
    y = (y - y.min())/(y.max()-y.min())
    return x, y

In [ ]:
x_train, y_train = load_data('SonyAIBORobotSurface1_IoC_DEV.txt')
x_test, y_test = load_data('SonyAIBORobotSurface1_IoC_FINAL_TEST.txt')

# MLP final

In [ ]:
def build_model():
    x = Input(shape=(x_train.shape[1:]), name='InputLayer')
    ### CHANGE PARAMETERS HERE ###
    y = Dropout(0.1,name='InputLayerDropout')(x)
    y = Dense(16, activation='relu', name='Layer010Dense')(y) 
    y = Dropout(0.2,name='Layer010Dropout')(y)
    y = Dense(16, activation='relu', name='Layer020Dense')(y)
    y = Dropout(0.2,name='Layer020Dropout')(y)
    y = Dense(16, activation='relu', name='Layer030Dense')(y)
    y = Dropout(0.3,name='Layer030Dropout')(y)
    ### END OF CHANGE PARAMETERS ###
    out = Dense(1, activation='sigmoid', name='OutputLayer')(y)

    # Build model and compile the model
    model = Model(x, out)
    optimizer = keras.optimizers.Adam()
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model = build_model()
batch_size = 5
epochs = 50

In [ ]:
start = time.time()
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, 
                 validation_data=(x_test, y_test), verbose=1)
end = time.time()
log = pd.DataFrame(hist.history) 
print('Training complete in', round(end-start), 'seconds')

In [ ]:
log[['acc', 'val_acc']].plot()
result = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Validation accuracy is', result[1])

# Save and reload the model
## Save

In [ ]:
# Save the model to a JSON file
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
# Save the model's weights to an HDF5 file
model.save_weights('model.h5')
print('Model saved')

## Load

In [ ]:
json_file = open('model.json', 'r')
loaded_json_model = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_json_model)
# load weights into new model
loaded_model.load_weights('model.h5')
print('Model loaded from file')

## Compile and use the loaded model

In [ ]:
optimizer = keras.optimizers.Adam()
loaded_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
result = loaded_model.evaluate(x_test, y_test, batch_size=batch_size)
print('Loaded model: validation accuracy is', result[1])
y_probability = loaded_model.predict_on_batch(x_test)
y_predicted_class = np.round(y_probability).flatten()
print('Some of the test results:')
print('True', y_test[:23])
print('Pred', y_predicted_class[:23])